In [1]:
#pip install --upgrade mplfinance

In [2]:
import pandas as pd

In [3]:
file = "./data/split/GBPUSD/weekly/GBPUSD_20190915_W_1401.csvx"
df = pd.read_csv(file)
df

,Unnamed: 0,time,open,high,low,close,symbol,minute,hour,weekday,week,month,year,day,RSI,SMA
0,0,9/09/2019 0:00,1.10232,1.10257,1.10232,1.10256,GBPUSD,0,0,0,37,9,2019,9,36.907857,1.103040
1,1,9/09/2019 0:05,1.10256,1.10256,1.10248,1.10249,GBPUSD,5,0,0,37,9,2019,9,34.539244,1.103015
2,2,9/09/2019 0:10,1.10249,1.10250,1.10249,1.10249,GBPUSD,10,0,0,37,9,2019,9,34.539244,1.102984
3,3,9/09/2019 0:15,1.10249,1.10256,1.10249,1.10249,GBPUSD,15,0,0,37,9,2019,9,34.539244,1.102954
4,4,9/09/2019 0:20,1.10249,1.10272,1.10249,1.10260,GBPUSD,20,0,0,37,9,2019,9,41.862141,1.102924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,1435,13/09/2019 23:35,1.10194,1.10199,1.10191,1.10194,EURUSD,35,23,4,38,9,2019,13,57.048845,1.101612
2876,1436,13/09/2019 23:40,1.10194,1.10196,1.10177,1.10181,EURUSD,40,23,4,38,9,2019,13,51.910159,1.101635
2877,1437,13/09/2019 23:45,1.10180,1.10180,1.10146,1.10168,EURUSD,45,23,4,38,9,2019,13,47.319934,1.101650
2878,1438,13/09/2019 23:50,1.10168,1.10196,1.10165,1.10190,EURUSD,50,23,4,38,9,2019,13,54.631350,1.101662


In [4]:
df['symbol'].unique()

array(['GBPUSD', 'EURUSD'], dtype=object)

In [5]:
from environments.forex_trading import tgym

In [6]:
import time 
import datetime

model_name = "./data/models/GBPUSD-week-" + datetime.datetime.now().strftime('%Y%m%d%H%M%S')
print(f'save model as {model_name}')

save model as ./data/models/GBPUSD-week-20211027083925


# Elegant RL, make sure elegantrl is in the root/ this folder. 
https://github.com/AI4Finance-Foundation/ElegantRL/tree/master/elegantrl

In [7]:
import torch
from elegantrl.run import *
from elegantrl.agent import *
import numpy as np
import time

In [8]:
args = Arguments()
args.agent = AgentPPO()
args.agent.cri_target = True
args.agent.lambda_entropy = 0.02
args.gamma = 0.997

In [9]:
parameters = {
    "observation_list":["open","high","low","close","minute","hour","weekday","RSI","SMA"],
    "stop_loss_max":200,
    "profit_taken_max":400    
}
#args.env = mytgym(df, **parameters)

args.env = tgym(df,env_config_file='./data/config/gdbusd-test-2.json')

self.action_dim: 2
self.state_dim: 24
self.action_space: Box(0.0, 3.0, (2,), float32)
self.observation_space: Box(-inf, inf, (23,), float32)
initial done:
observation_list:['open', 'high', 'low', 'close', 'minute', 'hour', 'day', 'RSI', 'SMA']
 assets:['GBPUSD' 'EURUSD']
 time serial: 10/09/2019 0:00 -> 9/09/2019 9:55 length: 1440


In [10]:
args.eval_gap = 2 ** 7
args.eval_times1 = 2 ** 0
args.eval_times2 = 2 ** 1

# Steps to take in training
args.break_step = 1e5 #int(5e5)


args.if_allow_break = False
args.random_seed = 312
args.gpu_id = 0
args.cwd = model_name
args.random_seed += int(args.gpu_id)
args.target_step = args.env.max_step * 1
args.worker_num = 1
# This needs to be same here and in loading
args.net_dim = 2**9

train_and_evaluate(args)

| Remove cwd: ./data/models/GBPUSD-week-20211027083925
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.


/opt/conda/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370116979/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


0  2.28e+03-4447.00 |
0  2.28e+03-4447.00 |-4447.00  600.0   1064   190 |    0.003952062.50   0.12  -0.50
| UsedTime:      28 | SavedDir: ./data/models/GBPUSD-week-20211027083925


# Test it 

In [17]:
args = Arguments(if_on_policy=True)
args.agent = AgentPPO()

# New for testing 
args.env = tgym(df,env_config_file='./data/config/gdbusd-test-2.json')
args.agent.if_use_cri_target = True

print()
print()

#load agent
try:
    state_dim = args.env.state_dim
    action_dim = args.env.action_dim

    agent = args.agent
    net_dim = 2**9


    cwd = model_name
    agent.init(net_dim, state_dim, action_dim)
    agent.save_or_load_agent(cwd=cwd, if_save=False) # False means that we load. 
    act = agent.act
    device = agent.device
    
except:
    raise ValueError('Fail to load agent!')
    
    
#test on the testing env
_torch = torch
state = args.env.reset()

actions = []
balances = list()  # the cumulative_return / initial_account
episode_returns = list()  
rewards = list()  

print('Max step:' + str(args.env.max_step))

with _torch.no_grad():
    for i in range(args.env.max_step):
    #for i in range(200):
        #if (i % 100 == 0):
            #print(round((i/args.env.max_step), 2)*100)
            #print(round((i/200), 2)*100)
        s_tensor = _torch.as_tensor((state,), device=device)
        a_tensor = act(s_tensor)  # action_tanh = act.forward()
        action = a_tensor.detach().cpu().numpy()[0]  # not need detach(), because with torch.no_grad() outside
        state, reward, done, _ = args.env.step(action)
        
        #print('Action: ' + str(action))
        #print('Reward: ' + str(reward))
        #print()
        
        actions.append(action)
        balances.append(args.env.balance)
        episode_return = args.env.balance / args.env.total_equity
        episode_returns.append(episode_return)
        rewards.append(reward)
        
        if done:
            break
            
            
args.env.close()
            
print('Test Finished!')
#return episode returns on testing data

self.action_dim: 2
self.state_dim: 24
self.action_space: Box(0.0, 3.0, (2,), float32)
self.observation_space: Box(-inf, inf, (23,), float32)
initial done:
observation_list:['open', 'high', 'low', 'close', 'minute', 'hour', 'day', 'RSI', 'SMA']
 assets:['GBPUSD' 'EURUSD']
 time serial: 10/09/2019 0:00 -> 9/09/2019 9:55 length: 1440


Max step:2879
Test Finished!


In [20]:
actions

[array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], dtype=float32),
 array([ 1., -1.], d

In [21]:
balances

[9900,
 9900,
 9890,
 9880,
 9870,
 9870,
 9860,
 9850,
 9840,
 9840,
 9840,
 9840,
 9840,
 9840,
 9840,
 9840,
 9840,
 9830,
 9820,
 9810,
 9810,
 9810,
 9810,
 9810,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9800,
 9700,
 9700,

In [22]:
episode_returns

[0.99,
 0.99,
 0.989,
 0.988,
 0.987,
 0.987,
 0.986,
 0.985,
 0.984,
 0.984,
 0.984,
 0.984,
 0.984,
 0.984,
 0.984,
 0.984,
 0.984,
 0.983,
 0.982,
 0.981,
 0.981,
 0.981,
 0.981,
 0.981,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98,
 0.98